 ## Imports and Parameters

In [1]:
import itertools
import random

import pandas as pd
import numpy as np
from sklearn.manifold import TSNE
import librosa

import IPython
import matplotlib.pyplot as plt
import seaborn as sns

import tensorflow.keras.backend as K
from tensorflow.keras import Model
from tensorflow.keras import Sequential
from tensorflow.keras.models import load_model
from tensorflow.keras.layers import Conv2D,Dense,MaxPool2D, Flatten,Lambda
from tensorflow.keras.layers import Dropout, Input, BatchNormalization
from tensorflow.keras.layers import SpatialDropout2D, GlobalMaxPool2D
from tensorflow.keras.optimizers import Adam, Nadam
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.regularizers import l2
from tensorflow.keras.initializers import he_normal
import tensorflow_addons as tfa

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [0]:
# Parameters
new_pairs = False
use_mfcc = True
data_scaled = True

n_seconds = 3
sample_freq = 16000 #LibriSpeech dataset was originaly sampled at 16kHz
downsampling = 2
n_mfcc = 30
mfcc_hop = 512

batch_size = 64
filters = 64
embedding_dimension = 1024
dropout_rate = 0.2

num_training_iter = 3000
evaluate_every_n_batches = 25
num_evaluation_tasks = 100 # how many one-shot tasks to validate on
n_shot = 1
k_way = 4  # how many classes for testing n-shot tasks

new_sample_freq = int(sample_freq / downsampling)
original_fragment_len = int(n_seconds * sample_freq)

if use_mfcc == False:
  input_length = int(new_sample_freq * n_seconds)
else:
  input_length_raw = int(new_sample_freq * n_seconds)
  input_length = (n_mfcc, 1+(input_length_raw//mfcc_hop), 1)

# Training/Testing data location
#training_set = 'train-clean-100'
training_set = 'sample'
test_set = 'dev-clean'
#test_set = 'sample-test'

## Get the Data


In [7]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [0]:
'''
from google_drive_downloader import GoogleDriveDownloader as gdd

gdd.download_file_from_google_drive(file_id='1XhQyzCQKwe6FH30tGdXVScGM45ZHJ--A',
                                    dest_path='./Data_samples.zip',
                                    unzip=False)
file_name = 'Data_samples.zip'

if downsampling == 2:
  gdd.download_file_from_google_drive(file_id='1GtYPhEdnfDWaGAqYHkR_oC9TD05bn8yb',
                                    dest_path='./Data_pairs.zip',
                                    unzip=False)
  file_name_2 = 'Data_pairs.zip'
elif downsampling == 4:
  gdd.download_file_from_google_drive(file_id='1ND29zxpHdoUjNY8k_mHtIuGLwwOadBSz',
                                    dest_path='./Data_pairs_4khz.zip',
                                    unzip=False)
  file_name_2 = 'Data_pairs_4khz.zip'
'''

In [8]:
!ls

drive  sample_data


In [9]:
! unzip -q -o '/content/drive/My Drive/DLAS/Data_pairs_mfcc_scaled.zip' -d /content/
! ls

drive		      test_diff_pairs.npy    X_test.npy   y_train.npy
sample_data	      train_alike_pairs.npy  X_train.npy
test_alike_pairs.npy  train_diff_pairs.npy   y_test.npy


In [10]:
! unzip -q -o '/content/drive/My Drive/DLAS/Data_samples.zip' -d /content/
! ls

Data	     test_alike_pairs.npy   train_diff_pairs.npy  y_test.npy
drive	     test_diff_pairs.npy    X_test.npy		  y_train.npy
sample_data  train_alike_pairs.npy  X_train.npy


#### Train set

In [11]:
# Get the data
train_tree = pd.read_csv('./Data/LibriSpeech/train/tree-'+ training_set +'.txt', 
                      names=['ID_reader', 'ID_chapter', 'Audio_file'],
                      sep='/',
                      header=None)
IPython.display.display(train_tree)

# Info of the dataset
n_readers = train_tree['ID_reader'].nunique()
print(f'Number of Readers: {n_readers}')

n_chapters = train_tree['ID_chapter'].nunique()
print(f'Number of Chapters: {n_chapters}')

min_chapters_reader = min(train_tree['ID_reader'].value_counts())
print(f'Min number of audio files in one reader: {min_chapters_reader}')

max_chapters_reader = max(train_tree['ID_reader'].value_counts())
print(f'Max number of audio files in one reader: {max_chapters_reader}')

Labels_ID_reader_train = list(train_tree['ID_reader'].unique())
print(f'Number of labels: {len(Labels_ID_reader_train)}')

,ID_reader,ID_chapter,Audio_file
0,103,1240,103-1240-0000.flac
1,103,1240,103-1240-0001.flac
2,103,1240,103-1240-0002.flac
3,103,1240,103-1240-0003.flac
4,103,1240,103-1240-0004.flac
...,...,...,...
9065,911,130578,911-130578-0016.flac
9066,911,130578,911-130578-0017.flac
9067,911,130578,911-130578-0018.flac
9068,911,130578,911-130578-0019.flac


Number of Readers: 80
Number of Chapters: 182
Min number of audio files in one reader: 60
Max number of audio files in one reader: 166
Number of labels: 80


In [12]:
# Listen to a file of train dataset
file_audio = './Data/LibriSpeech/train/'+ training_set +'/19/227/19-227-0015.flac'
y, sr = librosa.load(file_audio, sr=new_sample_freq)
IPython.display.Audio(data=y, rate=sr)

#### Test set

In [13]:
# Get the data
test_tree = pd.read_csv('./Data/LibriSpeech/val/tree-'+ test_set +'.txt', 
                      names=['ID_reader', 'ID_chapter', 'Audio_file'],
                      sep='/',
                      header=None)
IPython.display.display(test_tree)

# Info of the dataset
n_readers = test_tree['ID_reader'].nunique()
print(f'Number of Readers: {n_readers}')

n_chapters = test_tree['ID_chapter'].nunique()
print(f'Number of Chapters: {n_chapters}')

min_chapters_reader = min(test_tree['ID_reader'].value_counts())
print(f'Min number of audio files in one reader: {min_chapters_reader}')

max_chapters_reader = max(test_tree['ID_reader'].value_counts())
print(f'Max number of audio files in one reader: {max_chapters_reader}')

Labels_ID_reader_test = list(test_tree['ID_reader'].unique())
print(f'Number of labels: {len(Labels_ID_reader_test)}')

,ID_reader,ID_chapter,Audio_file
0,1272,128104,1272-128104-0000.flac
1,1272,128104,1272-128104-0001.flac
2,1272,128104,1272-128104-0002.flac
3,1272,128104,1272-128104-0003.flac
4,1272,128104,1272-128104-0004.flac
...,...,...,...
2698,8842,304647,8842-304647-0009.flac
2699,8842,304647,8842-304647-0010.flac
2700,8842,304647,8842-304647-0011.flac
2701,8842,304647,8842-304647-0012.flac


Number of Readers: 40
Number of Chapters: 97
Min number of audio files in one reader: 36
Max number of audio files in one reader: 101
Number of labels: 40


In [14]:
# Listen to a file of test dataset
file_audio = './Data/LibriSpeech/val/'+ test_set +'/1673/143396/1673-143396-0005.flac'
y, sr = librosa.load(file_audio, sr=new_sample_freq)
IPython.display.Audio(data=y, rate=sr)

## Create new pairs

#### Pairs of Train set

In [0]:
if new_pairs == True:
  # Get X and y from the training set
  X_train = []
  y_train = []

  for index, row in train_tree.iterrows():
          file_path = './Data/LibriSpeech/train/'+ training_set +'/' + str(row['ID_reader']) + '/' + str(row['ID_chapter']) + '/' + str(row['Audio_file'])
          data, _ = librosa.core.load(file_path, sr=new_sample_freq, res_type="kaiser_fast")
          n_segments = data.shape[0]//input_length_raw
          for j in range(n_segments):
              audio_segment = data[j*input_length_raw:(j+1)*input_length_raw]
                
              if use_mfcc == True:
                mfcc_segment = librosa.feature.mfcc(audio_segment,
                                                    sr=new_sample_freq,
                                                    n_mfcc=n_mfcc)
                mfcc_segment = mfcc_segment[:,:, np.newaxis]
                X_train.append(mfcc_segment)
              else:
                audio_segment = audio_segment[:, np.newaxis]
                X_train.append(audio_segment)
                
              y_train.append(row['ID_reader'])
  X_train = np.array(X_train)
  y_train = np.array(y_train)

  print(X_train.shape)
  print(y_train.shape)


In [0]:
if data_scaled == False:
  n_min = X_train.min()
  n_max = X_train.max()
  print(f'n_min {n_min}')
  print(f'n_max {n_max}')
  X_train = (X_train-n_min)/(n_max-n_min)
  X_train.shape

In [0]:
if new_pairs == True:
  X = np.copy(X_train)
  y = np.copy(y_train)
  categories = Labels_ID_reader_train.copy()

  # Alike Pairs
  train_alike_pairs = []
  len_class_pairs = {}
  for cat in categories:
      len_class_pairs.update({cat : 0})
      index_to_combine = []
      pairs = []
      for i in range(len(y)):
          if y[i] == cat:
              index_to_combine.append(i)
      pairs = list(itertools.combinations(index_to_combine,2))
      len_class_pairs[cat] += len(pairs)
      train_alike_pairs.extend(pairs)
      
  train_alike_pairs = np.array(train_alike_pairs)
  print(f'Train alike_pairs shape: {train_alike_pairs.shape}')
  assert (train_alike_pairs.shape[0] == sum(len_class_pairs.values())), 'Number of pairs is not what expected'


  # Different Pairs
  train_diff_pairs = []
  for j, key in enumerate(len_class_pairs):
      value = len_class_pairs.get(key)
      pairs_per_class = 38 + 2*(value//(len(categories)-1))
      #Obtain list A to combine
      index_to_combine_A = [r for r in range(len(y)) if y[r]==key]
      if j != (len(categories)-1):
          for index_comb in range(j+1, len(len_class_pairs)):
              key_comb = list(len_class_pairs)[index_comb]
              #Obtain list B to combine
              index_to_combine_B = [k for k in range(len(y)) if y[k]==key_comb]
              pairs = list(itertools.product(index_to_combine_A,index_to_combine_B))
              # Get random offset
              rnd_offset = random.randint(0,(len(pairs)-(pairs_per_class)))
              pairs = pairs[rnd_offset:(rnd_offset+pairs_per_class)]
              train_diff_pairs.extend(pairs)

  train_diff_pairs = np.array(train_diff_pairs)
  print(f'Train diff_pairs shape: {train_diff_pairs.shape}')

#### Pairs of Test set

In [0]:
if new_pairs == True:
  # Get X and y from the test set
  X_test = []
  y_test = []

  for index, row in test_tree.iterrows():
          file_path = './Data/LibriSpeech/val/'+ test_set +'/' + str(row['ID_reader']) + '/' + str(row['ID_chapter']) + '/' + str(row['Audio_file'])
          data, _ = librosa.core.load(file_path, sr=new_sample_freq, res_type="kaiser_fast")
          n_segments = data.shape[0]//input_length_raw
          for j in range(n_segments):
              audio_segment = data[j*input_length_raw:(j+1)*input_length_raw]
                
              if use_mfcc == True:
                mfcc_segment = librosa.feature.mfcc(audio_segment,
                                                    sr=new_sample_freq,
                                                    n_mfcc=n_mfcc)
                mfcc_segment = mfcc_segment[:,:, np.newaxis]
                X_test.append(mfcc_segment)
              else:
                audio_segment = audio_segment[:, np.newaxis]
                X_test.append(audio_segment)
                
              y_test.append(row['ID_reader'])
  X_test = np.array(X_test)
  y_test = np.array(y_test)

  print(X_test.shape)
  print(y_test.shape)

In [0]:
if data_scaled == False:
  n_min_t = X_test.min()
  n_max_t = X_test.max()
  print(f'n_min {n_min_t}')
  print(f'n_max {n_max_t}')
  X_test = (X_test-n_min_t)/(n_max_t-n_min_t)
  X_test.shape

In [0]:
if new_pairs == True:
  X = np.copy(X_test)
  y = np.copy(y_test)
  categories = Labels_ID_reader_test.copy()

  # Alike Pairs
  test_alike_pairs = []
  test_len_class_pairs = {}
  for cat in categories:
      test_len_class_pairs.update({cat : 0})
      index_to_combine = []
      pairs = []
      for i in range(len(y)):
          if y[i] == cat:
              index_to_combine.append(i)
      pairs = list(itertools.combinations(index_to_combine,2))
      test_len_class_pairs[cat] += len(pairs)
      test_alike_pairs.extend(pairs)
  test_alike_pairs = np.array(test_alike_pairs)
  print(f'Test alike_pairs shape: {test_alike_pairs.shape}')
  #assert (test_alike_pairs.shape[0] == sum(len_class_pairs.values())), 'Number of pairs is not what expected'


  # Different Pairs
  test_diff_pairs = []
  for j, key in enumerate(test_len_class_pairs):
      value = test_len_class_pairs.get(key)
      pairs_per_class = 2*(value//(len(categories)-1))
      #Obtain list A to combine
      index_to_combine_A = [r for r in range(len(y)) if y[r]==key]
      if j != (len(categories)-1):
          for index_comb in range(j+1, len(test_len_class_pairs)):
              key_comb = list(test_len_class_pairs)[index_comb]
              #Obtain list B to combine
              index_to_combine_B = [k for k in range(len(y)) if y[k]==key_comb]
              pairs = list(itertools.product(index_to_combine_A,index_to_combine_B))
              # Get random offset
              rnd_offset = random.randint(0,(len(pairs)-(pairs_per_class)))
              pairs = pairs[rnd_offset:(rnd_offset+pairs_per_class)]
              test_diff_pairs.extend(pairs)

  test_diff_pairs = np.array(test_diff_pairs)
  print(f'Test diff_pairs shape: {test_diff_pairs.shape}')

#### Save Created Pairs and Datasets

In [0]:
if new_pairs == True:
  # Save data from numpy arrays to .npy
  np.save('train_alike_pairs.npy', train_alike_pairs)
  np.save('train_diff_pairs.npy', train_diff_pairs)
  np.save('test_alike_pairs.npy', test_alike_pairs)
  np.save('test_diff_pairs.npy', test_diff_pairs)

  np.save('X_train.npy', X_train)
  np.save('y_train.npy', y_train)
  np.save('X_test.npy', X_test)
  np.save('y_test.npy', y_test)

## Load Pre-process Data

In [0]:
if new_pairs == False:
  # Load data from .npy files
  train_alike_pairs = np.load('train_alike_pairs.npy')
  train_diff_pairs = np.load('train_diff_pairs.npy')
  test_alike_pairs = np.load('test_alike_pairs.npy')
  test_diff_pairs = np.load('test_diff_pairs.npy')

  X_train = np.load('X_train.npy')
  y_train = np.load('y_train.npy')
  X_test = np.load('X_test.npy')
  y_test = np.load('y_test.npy')

## Data Shapes and Labels


In [23]:
# Number of audios per speaker on training set
df_counts_y = pd.DataFrame(y_train)
print('Training Set\n')
print('ID   || nº audios')
df_counts_y[0].value_counts()

Training Set

ID   || nº audios


696     458
87      456
196     454
226     454
250     453
       ... 
458     307
1034    284
1040    278
445     196
1183    162
Name: 0, Length: 80, dtype: int64

In [24]:
# Train Targets
labels_train = np.unique(y_train)
print(labels_train.shape)
labels_train

(80,)


array([  19,   26,   27,   32,   39,   40,   60,   78,   83,   87,   89,
        103,  118,  125,  150,  163,  196,  198,  200,  201,  211,  226,
        229,  233,  248,  250,  254,  289,  298,  302,  307,  311,  322,
        328,  332,  374,  403,  405,  412,  426,  441,  445,  446,  458,
        460,  481,  587,  625,  669,  696,  730,  831,  839,  887,  909,
        911, 1034, 1040, 1069, 1081, 1088, 1098, 1116, 1183, 1235, 1246,
       1263, 1334, 1355, 1363, 1447, 1455, 1502, 1553, 1578, 1594, 1624,
       1723, 1737, 1743])

In [25]:
# Number of audios per speaker on test_set
df_counts_y = pd.DataFrame(y_test)
print('Testing Set\n')
print('ID   || nº audios')
df_counts_y[0].value_counts()

Testing Set

ID   || nº audios


422     147
1673    142
2902    138
3576    138
3000    138
3170    137
2803    137
5338    134
6319    134
5536    134
652     134
3536    133
2078    132
174     132
1993    132
8842    131
2086    131
2412    129
3853    129
8297    126
7976    125
6295    125
1919    125
7850    125
2035    123
84      123
1988    122
6241    121
5694    121
1272    121
251     121
6345    120
777     119
2428    117
5895    116
3081    114
2277    114
1462    113
6313    111
3752    103
Name: 0, dtype: int64

In [26]:
# Test Targets
labels = np.unique(y_test)
print(labels.shape)
labels

(40,)


array([  84,  174,  251,  422,  652,  777, 1272, 1462, 1673, 1919, 1988,
       1993, 2035, 2078, 2086, 2277, 2412, 2428, 2803, 2902, 3000, 3081,
       3170, 3536, 3576, 3752, 3853, 5338, 5536, 5694, 5895, 6241, 6295,
       6313, 6319, 6345, 7850, 7976, 8297, 8842])

In [27]:
print(X_train.shape)
print(y_train.shape)

(33346, 32, 94, 1)
(33346,)


In [28]:
print(X_test.shape)
print(y_test.shape)

(5067, 32, 94, 1)
(5067,)


In [29]:
print(f'Train Alike Pairs shape: {train_alike_pairs.shape}')
print(f'Train Diff Pairs shape: {train_diff_pairs.shape}')

Train Alike Pairs shape: (7061574, 2)
Train Diff Pairs shape: (7062132, 2)


In [30]:
print(f'Test Alike Pairs shape: {test_alike_pairs.shape}')
print(f'Test Diff Pairs shape: {test_diff_pairs.shape}')

Test Alike Pairs shape: (320122, 2)
Test Diff Pairs shape: (321126, 2)


## Siamese Model


In [0]:
def get_siamese_model(input_shape, filters, dropout_rate, embedding_dimension):
    
    # Define the tensors for the two input images
    left_input = Input(input_shape)
    right_input = Input(input_shape)
    
    # Convolutional Neural Network
    model = Sequential()
    
    # 1 Block
    model.add(Conv2D(filters, (3, 3), 
                     activation='relu', 
                     padding='same',
                     input_shape=input_shape,
                     kernel_initializer=he_normal(), 
                     bias_initializer=he_normal(), 
                     kernel_regularizer=l2()))
    model.add(BatchNormalization())
    model.add(Conv2D(filters, (3, 3), 
                     activation='relu',
                     padding='same',
                     kernel_initializer=he_normal(),
                     bias_initializer=he_normal(), 
                     kernel_regularizer=l2()))
    model.add(BatchNormalization())
    model.add(MaxPool2D())
    model.add(SpatialDropout2D(dropout_rate))
    
    # 2 Block
    model.add(Conv2D(filters*2, (3, 3), 
                     activation='relu', 
                     padding='same',
                     kernel_initializer=he_normal(), 
                     bias_initializer=he_normal(), 
                     kernel_regularizer=l2()))
    model.add(BatchNormalization())
    model.add(Conv2D(filters*2, (3, 3), 
                     activation='relu',
                     padding='same',
                     kernel_initializer=he_normal(),
                     bias_initializer=he_normal(), 
                     kernel_regularizer=l2()))
    model.add(BatchNormalization())
    model.add(MaxPool2D())
    model.add(SpatialDropout2D(dropout_rate))
    
    # 3 Block
    model.add(Conv2D(filters*3, (3, 3), 
                     activation='relu', 
                     padding='same',
                     kernel_initializer=he_normal(), 
                     bias_initializer=he_normal(), 
                     kernel_regularizer=l2()))
    model.add(BatchNormalization())
    model.add(Conv2D(filters*3, (3, 3), 
                     activation='relu',
                     padding='same',
                     kernel_initializer=he_normal(),
                     bias_initializer=he_normal(), 
                     kernel_regularizer=l2()))
    model.add(BatchNormalization())
    model.add(MaxPool2D())
    model.add(SpatialDropout2D(dropout_rate))
    '''
    # 4 Block
    model.add(Conv2D(filters*4, (3, 3), 
                     activation='relu', 
                     padding='same',
                     kernel_initializer=he_normal(),
                     bias_initializer=he_normal(), 
                     kernel_regularizer=l2()))
    model.add(BatchNormalization())
    model.add(Conv2D(filters*4, (3, 3), 
                     activation='relu', 
                     padding='same',
                     kernel_initializer=he_normal(),
                     bias_initializer=he_normal(), 
                     kernel_regularizer=l2()))
    model.add(BatchNormalization())
    model.add(MaxPool2D())
    #model.add(Dropout(dropout_rate))
    model.add(SpatialDropout2D(dropout_rate))
    '''
    # Dense Block
    model.add(GlobalMaxPool2D())
    #model.add(Flatten())
    model.add(Dense(embedding_dimension, 
                    activation='sigmoid',
                    kernel_regularizer=l2(),
                    kernel_initializer=he_normal(),
                    bias_initializer=he_normal()))
    model.add(Dropout(dropout_rate))
    
    # Generate the encodings (feature vectors) for the two images
    encoded_l = model(left_input)
    encoded_r = model(right_input)
    
    # Add a customized layer to compute the absolute difference between the encodings
    L1_layer = Lambda(lambda tensors:K.abs(tensors[0] - tensors[1]))
    L1_distance = L1_layer([encoded_l, encoded_r])
    
    # Add a dense layer with a sigmoid unit to generate the similarity score
    prediction = Dense(1,activation='sigmoid')(L1_distance)
    
    # Connect the inputs with the outputs
    siamese_net = Model(inputs=[left_input,right_input],outputs=prediction)
    
    return siamese_net

In [32]:
model = get_siamese_model((X_train.shape[1], X_train.shape[2], 1),filters, dropout_rate, embedding_dimension)
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 32, 94, 1)]  0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, 32, 94, 1)]  0                                            
__________________________________________________________________________________________________
sequential (Sequential)         (None, 1024)         1013056     input_1[0][0]                    
                                                                 input_2[0][0]                    
__________________________________________________________________________________________________
lambda (Lambda)                 (None, 1024)         0           sequential[1][0]             

In [0]:
radam = tfa.optimizers.RectifiedAdam()
biCrossentropy = BinaryCrossentropy(label_smoothing=0.1)
model.compile(optimizer=radam,
              loss=biCrossentropy,
              metrics=['accuracy'])

## Aux. Functions


In [0]:
def get_batch(X, y, alike_pairs, diff_pairs, batch_size):
  '''
  Create Batch of n pairs, half alike class, half different class
  '''
  left_array = []  #[batch_size, 24000, 1]
  right_array = []   #[batch_size, 24000, 1]
  targets = []  #[batch_size, 1]
    

  # Select random samples of alike pairs
  alike_pairs_selected = random.sample(list(train_alike_pairs),k=(batch_size//2))
  for pair in alike_pairs_selected:
    left = pair[0]
    left = X[left]
    left_array.append(left)
    right = pair[1]
    right = X[right]
    right_array.append(right)

  # Select random samples of diff pairs
  diff_pairs_selected = random.sample(list(train_diff_pairs),k=(batch_size//2))
  for pair in diff_pairs_selected:
    left = pair[0]
    left = X[left]
    left_array.append(left)
    right = pair[1]
    right = X[right]
    right_array.append(right)

  # array to numpy
  left_array = np.array(left_array)
  right_array = np.array(right_array)

  # Targets
  targets.extend(np.ones(batch_size//2)) #1 for alike pairs
  targets.extend(np.zeros(batch_size//2)) #0 for diff pairs
  targets = np.array(targets)
  targets = targets[:, np.newaxis]

  #print(f'left_array: {left_array.shape}')
  #print(f'right_array: {right_array.shape}')
  #print(f'targets: {targets.shape}')

  return [left_array, right_array], targets

In [35]:
# Example of function get_batch for training process
inputs_example, targets_example = get_batch(X_train, y_train, train_alike_pairs, train_diff_pairs, batch_size)
inputs_example = np.array(inputs_example)
print(f'inputs_example shape {inputs_example.shape}')
print(f'targets_example shape {targets_example.shape}')

inputs_example shape (2, 64, 32, 94, 1)
targets_example shape (64, 1)


In [0]:
# Testing the model by one-shot task
def get_oneshot_pairs(X, y, n_shot, k_way, model, alike_pairs, diff_pairs, num_ev_tasks):
  if n_shot == 1:
    selected_labels_test = random.choices(list(labels), k=num_ev_tasks)
    left_array = []  #[num_evaluation_tasks*k_way, 24000, 1]
    right_array = []   #[num_evaluation_tasks*k_way, 24000, 1]
    targets = []  #[num_evaluation_tasks*k_way, 1]

    for label in selected_labels_test:
      # get indexes of y_test with same label
      index_y_same_label = [index for index in range(len(y)) if y[index]==label]
      # get alike pair
      test_sample = random.sample(index_y_same_label, k=2)
      '''
      test_sample = random.choice(index_yTest_same_label)
      alike_pairs_to_select = [list(pair) for pair in alike_pairs if pair[0]==test_sample]
      # prevent from empty lists
      while not alike_pairs_to_select:
        test_sample = random.choice(index_yTest_same_label)
        alike_pairs_to_select = [list(pair) for pair in alike_pairs if pair[0]==test_sample]
      selected_alike_pair = random.choice(alike_pairs_to_select)
      '''
      left = test_sample[0]
      right_alike = test_sample[1]
      # insert into output arrays
      left = X[left]
      left_array.append(left)
      right_alike = X[right_alike]
      right_array.append(right_alike)
      targets.append(1)

      # get k_way-1 samples from the others classes
      remain_labels = [label_aux for label_aux in labels if label_aux!=label]
      remain_labels = random.sample(remain_labels,k=(k_way-1))
      for x in remain_labels:
        y_remain_labels = [index for index in range(len(y)) if y[index]==x]
        selected_diff_samples = random.choice(y_remain_labels)
        #insert test sample on left array
        left_array.append(left)
        #insert diff sample on right array
        right_alike = X[selected_diff_samples]
        right_array.append(right_alike)
        targets.append(0)

    # array to numpy
    left_array = np.array(left_array)
    right_array = np.array(right_array)
    targets = np.array(targets)
    targets = targets[:, np.newaxis]

    #print(f'left_array: {left_array.shape}')
    #print(f'right_array: {right_array.shape}')
    #print(f'targets: {targets.shape}')

  elif n_shot > 1:
    # To Do
    print('Not Implemented Yet')
  return [left_array, right_array], targets

def test_oneshot(X, y, n_shot, k_way, model, alike_pairs, diff_pairs, num_ev_tasks):
  n_correct_random = 0
  n_correct = 0
  inputs, targets = get_oneshot_pairs(X, y, n_shot, k_way, model, alike_pairs, diff_pairs, num_ev_tasks)
  probs = model.predict(inputs)
  probs = np.array(probs)
  for j in range(targets.shape[0]//k_way):
    arr_probs = np.array(probs[j*k_way:(j+1)*k_way,:])
    arr_targ = np.array(targets[j*k_way:(j+1)*k_way,:])
    idx_target = np.argmax(arr_targ)
    if np.argmax(arr_probs) == idx_target:
      n_correct += 1
    if random.randint(0,k_way) == idx_target:
      n_correct_random += 1

  test_acc = (n_correct)/(num_ev_tasks)
  random_test_acc = (n_correct_random)/(num_ev_tasks)

  return test_acc, random_test_acc

In [37]:
# Example of function test_oneshot for testing process
test_acc, random_test_acc = test_oneshot(X_test, y_test, n_shot, k_way, model, test_alike_pairs, test_diff_pairs, num_evaluation_tasks)
print(test_acc, random_test_acc)

0.3 0.17


## Training Pipeline

In [0]:
# Training the model
losses = []
train_accuracies = []
test_accuracies = []
random_test_accuraccies = []
best = -1

print("Starting training process!")
for i in range(1, num_training_iter+1):
  [left_array, right_array], targets = get_batch(X_train, y_train, train_alike_pairs, train_diff_pairs, batch_size)
  inputs = [left_array, right_array]
  #print(f'inputs: {np.array(inputs).shape}')
  #print(f'targets: {targets.shape}')
  loss, train_acc = model.train_on_batch(inputs, targets)
  losses.append(loss)
  train_accuracies.append(train_acc)

  if i % evaluate_every_n_batches == 0:
    print('\n----------------------------------')
    print(f'Iteration number {i}')
    print(f'Training Loss {loss}')
    print(f'Training Accuracy {train_acc}')
    #Test n shot
    test_acc, random_test_acc = test_oneshot(X_test, y_test, n_shot, k_way, model, test_alike_pairs, test_diff_pairs, num_evaluation_tasks)
    test_accuracies.append(test_acc)
    random_test_accuraccies.append(random_test_acc)
    print(f'Test Accuracy {test_acc}')
    if test_acc > best:
      print(f'Current best accuracy: {test_acc}')
      print('Saving model...')
      #model.save('./saved_models/best_model', overwrite=True, include_optimizer=False)
      model.save('/content/drive/My Drive/DLAS/best_model', overwrite=True, include_optimizer=False)
      best = test_acc
    print('----------------------------------\n')

In [0]:
model.save('/content/drive/My Drive/DLAS/final_model', overwrite=True, include_optimizer=False)

In [0]:
# Ploting loss for train and acurracy for both train and test
for data in [losses, train_accuracies]:
    plt.figure(figsize=(10,10))
    plt.plot(data, linewidth=2, label='Train')
    smoothed_data = pd.Series.rolling(pd.Series(data), 100).mean()
    plt.plot(smoothed_data, linewidth=2, label='Smoothed')
    #plt.plot(model_history['val_'+data], linewidth=2, label='Valid')
    plt.legend(loc='upper right')
    #plt.title('Model '+ data)
    #plt.ylabel(data)
    plt.xlabel('Iteration')
    plt.show()

plt.figure(figsize=(10,10))
plt.plot(test_accuracies, linewidth=2, label='Test')
smoothed_data = pd.Series.rolling(pd.Series(test_accuracies), 10).mean()
plt.plot(smoothed_data, linewidth=2, label='Smoothed Test')
plt.plot(random_test_accuraccies, linewidth=2, label='Random')
smoothed_data = pd.Series.rolling(pd.Series(random_test_accuraccies), 10).mean()
plt.plot(smoothed_data, linewidth=2, label='Smoothed Random')
plt.legend(loc='upper right')
plt.title("{} way one-shot learning accuracy \n".format(k_way))
plt.ylabel('Accuracy')
plt.xlabel('Iteration x {}'.format(evaluate_every_n_batches))
plt.show()

## Test Models (k-way learning)

In [0]:
!ls drive/My\ Drive/DLAS

In [0]:
model_best_val = load_model('/content/drive/My Drive/DLAS/best_model/')
radam = tfa.optimizers.RectifiedAdam()
biCrossentropy = BinaryCrossentropy(label_smoothing=0.1)
model_best_val.compile(optimizer=radam,
                       loss='binary_crossentropy',
                       metrics=['accuracy'])

In [0]:
model_last = load_model('/content/drive/My Drive/DLAS/final_model/')
radam = tfa.optimizers.RectifiedAdam()
biCrossentropy = BinaryCrossentropy(label_smoothing=0.1)
model_last.compile(optimizer=radam,
                   loss='binary_crossentropy',
                   metrics=['accuracy'])

In [0]:
# Test best model
test_k_way_acc = []
random_k_way_acc = []
for k in range(2,len(labels)):
  test_k_way, random_test_k_way = test_oneshot(X=X_test, y=y_test, n_shot=n_shot, k_way=k, model=model_best_val, alike_pairs=test_alike_pairs, diff_pairs=test_diff_pairs, num_ev_tasks=num_evaluation_tasks)
  test_k_way_2, random_test_k_way = test_oneshot(X=X_test, y=y_test, n_shot=n_shot, k_way=k, model=model_best_val, alike_pairs=test_alike_pairs, diff_pairs=test_diff_pairs, num_ev_tasks=num_evaluation_tasks)
  print(f'test {k}-way: {test_k_way} ~ {test_k_way_2}')
  test_k_way_def = (test_k_way+test_k_way_2)/2
  test_k_way_acc.append(test_k_way_def)
  random_k_way_acc.append(random_test_k_way)

# Visualize test of best model
plt.figure(figsize=(10,10))
plt.plot(test_k_way_acc, linewidth=2, label='Siamese_model')
smoothed_data = pd.Series.rolling(pd.Series(test_k_way_acc), 3).mean()
plt.plot(smoothed_data, linewidth=2, label='Smoothed')
plt.plot(random_k_way_acc, linewidth=2, label='Random_baseline')
smoothed_data = pd.Series.rolling(pd.Series(random_k_way_acc), 3).mean()
plt.plot(smoothed_data, linewidth=2, label='Smoothed')
plt.title(f'1-shot K-way classification accuracy over {num_evaluation_tasks} tasks')
plt.ylabel('Accuracy')
plt.xlabel('K-way')
plt.show()

In [0]:
# Test last model
test_k_way_acc = []
random_k_way_acc = []
for k in range(2,len(labels)):
  test_k_way, random_test_k_way = test_oneshot(X=X_test, y=y_test, n_shot=n_shot, k_way=k, model=model_last, alike_pairs=test_alike_pairs, diff_pairs=test_diff_pairs, num_ev_tasks=num_evaluation_tasks)
  test_k_way_2, random_test_k_way = test_oneshot(X=X_test, y=y_test, n_shot=n_shot, k_way=k, model=model_last, alike_pairs=test_alike_pairs, diff_pairs=test_diff_pairs, num_ev_tasks=num_evaluation_tasks)
  print(f'test {k}-way: {test_k_way} ~ {test_k_way_2}')
  test_k_way_def = (test_k_way+test_k_way_2)/2
  test_k_way_acc.append(test_k_way_def)
  random_k_way_acc.append(random_test_k_way)

# Visualize test of last model
plt.figure(figsize=(10,10))
plt.plot(test_k_way_acc, linewidth=2, label='Siamese_model')
smoothed_data = pd.Series.rolling(pd.Series(test_k_way_acc), 3).mean()
plt.plot(smoothed_data, linewidth=2, label='Smoothed')
plt.plot(random_k_way_acc, linewidth=2, label='Random_baseline')
smoothed_data = pd.Series.rolling(pd.Series(random_k_way_acc), 3).mean()
plt.plot(smoothed_data, linewidth=2, label='Smoothed')
plt.title(f'1-shot K-way classification accuracy over {num_evaluation_tasks} tasks')
plt.ylabel('Accuracy')
plt.xlabel('K-way')
plt.show()

## Embedding Space Visualization

In [0]:
# Extract encoder network
encoder = model_best_val.layers[2]

#Compile model
radam = tfa.optimizers.RectifiedAdam()
biCrossentropy = BinaryCrossentropy(label_smoothing=0.1)
encoder.compile(optimizer=radam,
                   loss='binary_crossentropy',
                   metrics=['accuracy'])

In [0]:
n_speakers = 10
k_samples = 10

In [0]:
#Get Data
X_samples = []
y_labels = []

speakers_labels = random.sample(list(labels_train), k=n_speakers)

for label in speakers_labels:
  # get indexes of y_train with same label aka. same speaker
  index_same_speaker = [index for index in range(len(y_train)) if y_train[index]==label]
  # get random samples indexes of speaker
  rnd_samples_indexes = random.sample(index_same_speaker, k=k_samples)
  X_sample = X_train[rnd_samples_indexes]
  X_samples.extend(X_sample)
  # Add k_samples times the label value to y_labels
  y_labels += k_samples * [label]

X_samples = np.array(X_samples)
y_labels = np.array(y_labels)

print(f'X_samples shape {X_samples.shape}')
print(f'y_labels shape {y_labels.shape}')

In [0]:
# Get Embeddings
embeddings = encoder.predict(X_samples)
print(f'embedings shape {embeddings.shape}')

In [0]:
# Dimensionality Reduction
tsne = TSNE(n_components=2,perplexity=30.0, learning_rate=100, early_exaggeration=24)

tsne_embeddings = tsne.fit_transform(embeddings)
print(f't-SNE embedings shape {tsne_embeddings.shape}')

In [0]:
Z = np.hstack([tsne_embeddings, y_labels[:, np.newaxis]])

plt.figure(figsize=(12,12))
ax = plt.axes()
ax.scatter(Z[:, 0], Z[:, 1], c=[np.unique(Z[:, 2]).tolist().index(i) for i in Z[:, 2]], 
           marker='^', cmap='tab10')
ax.axes.set_xticklabels([])
ax.axes.set_yticklabels([])
plt.title('Embeding Space Visualization (t-SNE)')
plt.grid()
plt.show()